# Chapter 3: Negative Prompts

---

**Lesson:**

Very similar to building strong prompts is crafting strong negative prompts. The purpose here is to indicate what you *don't* want to see in your image. We do this by providing additional prompts that follow the same format and concept as "positive" prompts, but the purpose is *opposite*.

In [ ]:
%%capture
# Install dependencies
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet "pillow>=9.5,<10"

# Python Built-Ins:
import base64
import io
import json
import os
import sys

# External Dependencies:
import boto3
from PIL import Image

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

modelId = "stability.stable-diffusion-xl"

## How to add a negative prompt and why

It should be noted that images can still be generated even without a negative prompt (else we wouldn't have created anything of value in our previous chapters). However, we can enhance our images by clarifying what we don't want. The reason this is important can be exemplified by the below.


In [ ]:
prompt = "a person without a hat"

Intuitively, we would expect the above prompt to generate an image of a person that is not wearing a hat.

In [ ]:
request = json.dumps({
    "text_prompts": (
        [{"text": prompt}]
    )
})

response = boto3_bedrock.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

print(response_body["result"])
base_64_img_str = response_body["artifacts"][0].get("base64")
print(f"{base_64_img_str[0:80]}...")

os.makedirs("data", exist_ok=True)
eg3_0a = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
eg3_0a.save("data/eg3_0a.png")
eg3_0a

What happened here? We specified a person *without* a hat, and yet that is not what we got. Herein lies the importance of negative prompting: Stable Diffusion *expects* us to use negative prompting to specify what we do not want in our image. There is no promise that a given element (such as a hat) will appear in your generated image, but to truly eliminate or at least greatly reduce the probability of that element appearing, we must use negative prompts.

Now, let's try the above example but with negative prompts.

In [ ]:
prompt = "a person"
negative_prompts = [
    "hat"
]

Notice the *weight* parameter added in the below. We have kept this parameter hidden until now as the default (1.0) was sufficient for our positive prompts. However, to designate a prompt as negative, we must assign it a weight of -1.0. More to come in Chapter 4 on taking further advantage of weights.

In [ ]:
request = json.dumps({
    "text_prompts": (
        [{"text": prompt}]
        + [{"text": negprompt, "weight": -1.0} for negprompt in negative_prompts]
    )
})

response = boto3_bedrock.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

print(response_body["result"])
base_64_img_str = response_body["artifacts"][0].get("base64")
print(f"{base_64_img_str[0:80]}...")

os.makedirs("data", exist_ok=True)
eg3_0b = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
eg3_0b.save("data/eg3_0b.png")
eg3_0b

## Example:

**Example 3.1 - A positive image**

Let's experiment more with negative prompts by creating a crisp image that changes with every generation (i.e., seed is 0), but remains sunny without a cloud in the sky.

In [ ]:
prompt = "a sunny day, green grass, smiling person, crisp image"
negative_prompts = [
    "clouds",
    "storms",
    "sad",
    "poor background details",
]

In [ ]:
request = json.dumps({
    "text_prompts": (
        [{"text": prompt}]
        + [{"text": negprompt, "weight": -1.0} for negprompt in negative_prompts]
    )
})

response = boto3_bedrock.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

print(response_body["result"])
base_64_img_str = response_body["artifacts"][0].get("base64")
print(f"{base_64_img_str[0:80]}...")

os.makedirs("data", exist_ok=True)
eg3_1 = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
eg3_1.save("data/eg3_1.png")
eg3_1

## Exercises:

We will now incorporate negative prompting into some previous examples to see how the outcome is affected.

**Exercise 3.1 - Replicating stories**

Using proper formatting, the included parameters, detailed descriptions, and negative prompting, generate an image based a scene in a book you read or a movie you saw recently.

In [ ]:
prompt = "INSERT PROMPT"
negative_prompts = [
    "INSERT NEGATIVE PROMPT",
    "INSERT NEGATIVE PROMPT"
]

cfg_scale = 
seed = 
steps = 
style_preset =   # (e.g. photographic, digital-art, cinematic, ...)
clip_guidance_preset =  # (e.g. FAST_BLUE FAST_GREEN NONE SIMPLE SLOW SLOWER SLOWEST)
sampler =  # (e.g. DDIM, DDPM, K_DPMPP_SDE, K_DPMPP_2M, K_DPMPP_2S_ANCESTRAL, K_DPM_2, K_DPM_2_ANCESTRAL, K_EULER, K_EULER_ANCESTRAL, K_HEUN, K_LMS)
width = 

In [ ]:
request = json.dumps({
    "text_prompts": (
        [{"text": prompt}]
        + [{"text": negprompt, "weight": -1.0} for negprompt in negative_prompts]
    ),
    "cfg_scale": cfg_scale,
    "seed": seed,
    "steps": steps,
    "style_preset": style_preset,
    "clip_guidance_preset": clip_guidance_preset,
    "sampler": sampler,
    "width": width
})

response = boto3_bedrock.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

print(response_body["result"])
base_64_img_str = response_body["artifacts"][0].get("base64")
print(f"{base_64_img_str[0:80]}...")

os.makedirs("data", exist_ok=True)
ex3_1 = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
ex3_1.save("data/ex3_1.png")
ex3_1

**Exercise 3.2 - Capturing an image**

See if you can recreate an image of any particular scene in your life, be it your office, park, house, etc. Think about the details required to make this come to life.

In [ ]:
prompt = "INSERT PROMPT"
negative_prompts = [
    "INSERT NEGATIVE PROMPT",
    "INSERT NEGATIVE PROMPT"
]

cfg_scale = 
seed = 
steps = 
style_preset =   # (e.g. photographic, digital-art, cinematic, ...)
clip_guidance_preset =  # (e.g. FAST_BLUE FAST_GREEN NONE SIMPLE SLOW SLOWER SLOWEST)
sampler =  # (e.g. DDIM, DDPM, K_DPMPP_SDE, K_DPMPP_2M, K_DPMPP_2S_ANCESTRAL, K_DPM_2, K_DPM_2_ANCESTRAL, K_EULER, K_EULER_ANCESTRAL, K_HEUN, K_LMS)
width = 

In [ ]:
request = json.dumps({
    "text_prompts": (
        [{"text": prompt}]
        + [{"text": negprompt, "weight": -1.0} for negprompt in negative_prompts]
    ),
    "cfg_scale": cfg_scale,
    "seed": seed,
    "steps": steps,
    "style_preset": style_preset,
    "clip_guidance_preset": clip_guidance_preset,
    "sampler": sampler,
    "width": width
})

response = boto3_bedrock.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

print(response_body["result"])
base_64_img_str = response_body["artifacts"][0].get("base64")
print(f"{base_64_img_str[0:80]}...")

os.makedirs("data", exist_ok=True)
ex3_2 = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
ex3_2.save("data/ex3_2.png")
ex3_2